<a href="https://colab.research.google.com/github/sanaharora/sanaharora/blob/main/DataNeuron_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-auth

In [3]:
import google.colab.auth

In [5]:
google.colab.auth.authenticate_user()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd

import re
from tqdm import tqdm

import collections

from sklearn.cluster import KMeans

from nltk.stem import WordNetLemmatizer  # For Lemmetization of words
from nltk.corpus import stopwords  # Load list of stopwords
from nltk import word_tokenize # Convert paragraph in tokens

import pickle
import sys

from gensim.models import word2vec # For represent words in vectors
import gensim

In [7]:
file_path = '/content/drive/MyDrive/DataNeuron_Text_Similarity.csv'
text_data = pd.read_csv(file_path)
print("Shape of text_data : ", text_data.shape)
text_data.head(3)

Shape of text_data :  (3000, 2)


,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...


In [8]:
text_data.isnull().sum() # Check if text data have any null values

text1    0
text2    0
dtype: int64

In [9]:
# Preprocessing
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
!pip install nltk

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
# Combining all the above stundents

preprocessed_text1 = []

# tqdm is for printing the status bar

for sentance in tqdm(text_data['text1'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text1.append(sent.lower().strip())

100%|██████████| 3000/3000 [02:26<00:00, 20.47it/s]


In [15]:
# Merging preprocessed_text1 in text_data

text_data['text1'] = preprocessed_text1
text_data.head(3)

,text1,text2
0,broadband challenges tv viewing number europea...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested drug find rap mogul marion s...,amnesty chief laments war failure the lack of ...
2,player burn worries robinson england coach and...,hanks greeted at wintry premiere hollywood sta...


In [16]:
# Combining all the above stundents
from tqdm import tqdm
preprocessed_text2 = []

# tqdm is for printing the status bar
for sentance in tqdm(text_data['text2'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text2.append(sent.lower().strip())

100%|██████████| 3000/3000 [02:24<00:00, 20.70it/s]


In [17]:
# Merging preprocessed_text2 in text_data

text_data['text2'] = preprocessed_text2

text_data.head(3)

,text1,text2
0,broadband challenges tv viewing number europea...,gardener wins double glasgow britain jason gar...
1,rap boss arrested drug find rap mogul marion s...,amnesty chief laments war failure lack public ...
2,player burn worries robinson england coach and...,hanks greeted wintry premiere hollywood star t...


In [18]:
def word_tokenizer(text):
            #tokenizes and stems the text
            tokens = word_tokenize(text)
            lemmatizer = WordNetLemmatizer()
            tokens = [lemmatizer.lemmatize(t) for t in tokens]
            return tokens

In [18]:
import gensim

In [19]:

wordmodelfile='/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz'
wordmodel= gensim.models.KeyedVectors.load_word2vec_format(wordmodelfile, binary=True)

In [20]:
!pip install nltk

In [21]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
from nltk.tokenize import word_tokenize

In [24]:
!pip install --upgrade gensim

In [25]:
from gensim.models import KeyedVectors

In [27]:
similarity = [] # List for store similarity score



for ind in text_data.index:

        s1 = text_data['text1'][ind]
        s2 = text_data['text2'][ind]

        if s1==s2:
                 similarity.append(0.0) # 0 means highly similar

        else:

            s1words = word_tokenize(s1)
            s2words = word_tokenize(s2)



            vocab = wordmodel.key_to_index #the vocabulary considered in the word embeddings

            if len(s1words and s2words)==0:
                    similarity.append(1.0)

            else:

                for word in s1words.copy(): #remove sentence words not found in the vocab
                    if (word not in vocab):


                            s1words.remove(word)


                for word in s2words.copy(): #idem

                    if (word not in vocab):

                            s2words.remove(word)


                similarity.append((1-wordmodel.n_similarity(s1words, s2words)))

In [29]:
import pandas as pd

In [33]:
# Get Unique_ID and similarity

final_score = pd.DataFrame({'Similarity_score':similarity})
final_score.head(3)

,Similarity_score
0,0.221688
1,0.240246
2,0.169012


In [34]:
final_score.to_csv('final_score.csv',index=False)